### Engineering

In [1]:
import torch

In [2]:
from torch import nn
import torch.nn.functional as F

In [ ]:
job queue
syncronization queue
handshake queue

In [ ]:
kubeproxy, kubelet, 

In [ ]:
(current_local + 1) % local_world_size

In [ ]:
mine_dir = linear_probe[..., 2]
theirs_dir = linear_probe[..., 1]

In [ ]:
mine_vs_theirs_dir = mine_dir - theirs_dir

In [ ]:
_, cache = model.run_with_cache(board_history)

In [3]:
from transformer_lens.utils import get_act_name

In [ ]:
mlp0 = model.blocks[0].mlp
ln0 = model.blocks[0].ln1

In [ ]:
name_tokens = model.to_tokens(names)

In [ ]:
embeddings = model.W_U[:, name_tokens]

In [ ]:
resid_after_mlp0 = embeddings + mlp0(ln0(embeddings))

In [ ]:
for layer_idx in range(n_layers):
    for head_idx in range(n_heads):
        W_OV = model.W_O[layer_idx, head_idx] @ model.W_O[layer_idx, head_idx]
        output = resid_after_mlp0 @ W_OV
        logits = model.unembed(model.ln_final(output))

In [4]:
import torch.distributed.rpc as rpc
import torch.distributed as dist

In [5]:
import torch.multiprocessing as mp

In [6]:
import os

In [ ]:
def init_dist(rank, world_size):
    addr = os.getenv("MASTER_ADDR")
    port = os.getenv("MASTER_PORT")
    
    init_method = f"tcp://{addr}:{port}"
    
    rpc.init_rpc(
        name=AGENT_NAME.format(rank),
        rank=rank,
        world_size=world_size,
        init_method=init_method,
    )

In [ ]:
for rank in range(world_size):
    p = mp.Process(
        target=init_dist,
        args=(rank, world_size)
    )

In [7]:
from typing import overload

In [9]:
from typing import List

In [8]:
@overload
def getitem(x: str) -> str:
    pass

In [10]:
@overload
def getitem(x: List[int]) -> int:
    pass

In [ ]:
name_tokens = model.to_tokens(names)

In [ ]:
mlp0 = model.blocks[0].mlp0
ln1 = model.blocks[0].ln1

In [ ]:
embeddings = model.embed(name_tokens)
resid_after_mlp0 = embeddings + mlp0(ln1(embeddings))

In [ ]:
for layer_idx in range(n_layers):
    for head_idx in range(n_heads):
        W_OV = model.W_O[layer_idx, head_idx] @ model.W_V[layer_idx, head_idx]
        output = resid_after_mlp0 @ W_OV
        logits = model.unembed(model.ln_final(output))
        
        top_tokens = torch.argmax(logits, dim=-1)

In [ ]:
clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

In [ ]:
_, clean_cache = model.run_with_cache(clean_tokens)
_, corrupted_cache = model.run_with_cache(corrupted_tokens)

In [ ]:
seq_len = clean_tokens.shape[-1]

In [11]:
def corrupt_activations(resid, hook, corrupted_cache, position_idx):
    resid[:, position_idx] = corrupted_cache[hook.name][:, position_idx]
    return resid

In [12]:
from functools import partial

In [ ]:
def compute_logit_diff(resid, correct_token, incorrect_token):
    final_resid = resid[:, -1, :]
    return final_resid[:, correct_token] - final_resid[:, incorrect_token]

In [ ]:
for layer_idx in range(n_layers):
    for position_idx in range(seq_len):
        hook_name = get_act_name("resid_pre", layer_idx)
        hook_func = partial(
            corrupt_activations,
            corrupted_cache=corrupted_cache,
            position_idx=position_idx
        )
        
        patched_logits = model.run_with_hooks(
            clean_tokens,
            fwd_hooks=[(hook_name, hook_func)]
        )